# Visualization graphs and results for Typhon

### Load Librairies

In [2]:
%matplotlib inline 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from pathlib import Path

### Put the path and select experiment to load

In [3]:
# Make sure the path is correct, i.e. change according which experiment we want
root_path = Path('results')

experiments = [path.stem for path in root_path.iterdir()]
exp_radiobuttons = widgets.RadioButtons(options=experiments)
display(exp_radiobuttons)

RadioButtons(options=('prostate_0',), value='prostate_0')

### Use buttons to select the datasets, the metrics and other stuff

In [4]:
path = root_path / exp_radiobuttons.value / 'run_plot'
metrics_plot = pd.read_csv(path / 'metrics.csv')
 
datasets_checkboxes = [widgets.Checkbox(value=True, description=label) for label in pd.unique(metrics_plot['dataset'])]
datasets_vbox = widgets.VBox(children=datasets_checkboxes)
 
types_checkboxes = [widgets.Checkbox(value=True, description=label) for label in pd.unique(metrics_plot['type'])]
types_vbox = widgets.VBox(children=types_checkboxes)

fe_checkboxes = [widgets.Checkbox(value=True, description=label) for label in pd.unique(metrics_plot['feature_extractor'])]
fe_vbox = widgets.VBox(children=fe_checkboxes)

metrics_checkboxes = [widgets.Checkbox(value=True if label=='loss' or label=='auc' else False, description=label) for label in pd.unique(metrics_plot['metric'])]
metrics_vbox = widgets.VBox(children=metrics_checkboxes)

vboxes = [datasets_vbox, types_vbox, fe_vbox, metrics_vbox]
output = widgets.HBox(children=vboxes)
display(output)

### Display the selected plots, and export it

In [ ]:
# Get the filter from the checkboxes
chosen_datasets = []
chosen_types = []
chosen_fe = []
chosen_metrics = []

for checkbox in datasets_checkboxes:
    if checkbox.value == True:
        chosen_datasets = chosen_datasets + [checkbox.description]
        
for checkbox in types_checkboxes:
    if checkbox.value == True:
        chosen_types = chosen_types + [checkbox.description]
        
for checkbox in fe_checkboxes:
    if checkbox.value == True:
        chosen_fe = chosen_fe + [checkbox.description]
        
for checkbox in metrics_checkboxes:
    if checkbox.value == True:
        chosen_metrics = chosen_metrics + [checkbox.description]

# Create figure
nplots = len(chosen_datasets)*len(chosen_types)*len(chosen_fe)*len(chosen_metrics)
fig, axes = plt.subplots(int(np.ceil(nplots / 2)), 3, figsize=(11,11.7/10*(nplots)), squeeze=False)

# Enable axes while plotting
for row in axes:
    for axis in row:
        axis.set_axis_off()
pos = 0

# Dict to use pretty print on the graphs
pretty_print = {
    'auc': 'AUC',
    'loss': 'Loss',
    'accuracy': 'Accuracy',
    'precision': 'Precision',
    'recall': 'Recall',
    'f1score': 'F1 Score',
    'specificity': 'Specificity',
    'trained': 'Training',
    'specialized': 'Specialization',
}
        
# Create the plots
for dset_name in chosen_datasets:
    dset_metrics_plot = metrics_plot[metrics_plot['dataset'] == dset_name]
    
    # For specialization and trained
    for type_name in chosen_types:
        type_plot = dset_metrics_plot[dset_metrics_plot['type'] == type_name]
        
        # For frozen and unfrozen feature extractor
        for fe in chosen_fe:
            fe_plot = type_plot[type_plot['feature_extractor'] == fe]

            # Plots valset and trainset performance
            for idx, metric_name in enumerate(chosen_metrics):
                metric_name_plot = fe_plot[fe_plot['metric'] == metric_name]
                if metric_name_plot.empty: continue
                legends = []
                results_test = {}
                # Display the axes
                axes[int(np.floor(pos / 2)), pos % 2].set_axis_on()

                # For Split (train, validation and test)
                for split in pd.unique(metric_name_plot['split']):
                    split_plot = metric_name_plot[metric_name_plot['split'] == split]
                    if split == 'test':
                        results_test[metric_name] = split_plot['value'].values[0]
                    else:    
                        # Cast the Pandas Series to a Python list
                        points = [point if point <= 1 else point for point in split_plot['value']]
                        axes[int(np.floor(pos / 2)), pos % 2].plot(points)
                        legends.append(f"{'T' if split == 'train' else 'V'}")

                # Add legend, title and axes name
                if type_plot['epoch'].empty: continue
                number_epoch = max(type_plot['epoch'])
                axes[int(np.floor(pos / 2)), pos % 2].legend(legends, loc='upper left', fontsize=5)
                axes[int(np.floor(pos / 2)), pos % 2].set_title(f"{dset_name}, {pretty_print[type_name]}, {pretty_print[metric_name]}")
                axes[int(np.floor(pos / 2)), pos % 2].set_xlabel('Epochs')
                axes[int(np.floor(pos / 2)), pos % 2].set_ylabel(pretty_print[metric_name])

                # Print results of test
                try:
                    axes[int(np.floor(pos / 2)), 2].axis('off')
                    axes[int(np.floor(pos / 2)), 2].text(0, (pos % 2)*0.4, f"{pretty_print[metric_name]}={results_test[metric_name]}")
                except KeyError:
                    pass
                pos += 1
        
# Automatic padding between plots
fig.tight_layout()
plt.savefig(path / f"{exp_radiobuttons.value}.pdf")
plt.show()